In [2]:
pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 16.4 MB/s eta 0:00:00


In [26]:
import pandas as pd
import joblib, json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.metrics import classification_report, accuracy_score

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv("/content/drive/MyDrive/법률모델링/divorce_topic_labeled_augmented.csv", encoding="utf-8-sig")

In [29]:
TEXT_COL = "input"
INTENT_COL = "law_intent"
TOPIC_COL = "topic_name"

print(f"📁 데이터 로드 완료: {len(df)}개 문장")

# ==============================
# 2️⃣ 공통 FeatureUnion (Word + Char)
# ==============================
combined_features = FeatureUnion([
    ("word", TfidfVectorizer(
        analyzer="word",
        ngram_range=(1,2),
        max_features=5000
    )),
    ("char", TfidfVectorizer(
        analyzer="char_wb",
        ngram_range=(3,6),
        min_df=2
    )),
])

# ==============================
# 3️⃣ 학습 함수
# ==============================
def train_and_eval(texts, labels, label_name, save_path):
    # 💡 각 모델별 FeatureUnion 독립 선언
    combined_features = FeatureUnion([
        ("word", TfidfVectorizer(
            analyzer="word",
            ngram_range=(1,2),
            max_features=5000
        )),
        ("char", TfidfVectorizer(
            analyzer="char_wb",
            ngram_range=(3,6),
            min_df=2
        )),
    ])

    X_train, X_test, y_train, y_test = train_test_split(
        texts, labels, test_size=0.15, random_state=42, stratify=labels
    )

    pipe = Pipeline([
        ("features", combined_features),
        ("clf", LinearSVC(C=1.2, class_weight="balanced")),
    ])

    print(f"\n🧠 [{label_name}] 모델 학습 중...")
    pipe.fit(X_train, y_train)

    preds = pipe.predict(X_test)
    acc = accuracy_score(y_test, preds)
    print(f"✅ {label_name} 정확도: {acc:.4f}")
    print(classification_report(y_test, preds))

    joblib.dump(pipe, save_path)
    print(f"💾 {save_path} 저장 완료 ✅")

    return pipe


# ==============================
# 4️⃣ 모델 학습
# ==============================
intent_pipe = train_and_eval(df[TEXT_COL], df[INTENT_COL], "law_intent", "/content/drive/MyDrive/intent_model_hybrid.joblib")
topic_pipe  = train_and_eval(df[TEXT_COL], df[TOPIC_COL ],  "topic_name",  "/content/drive/MyDrive/topic_model_hybrid.joblib")

📁 데이터 로드 완료: 1332개 문장

🧠 [law_intent] 모델 학습 중...
✅ law_intent 정확도: 0.7450
              precision    recall  f1-score   support

    가능 여부 판단       0.81      0.86      0.83        50
    개념·범위·기준       0.77      0.62      0.69        16
       금액·산정       0.71      0.79      0.75        19
       기간·시효       0.82      0.78      0.80        18
          기타       0.83      0.83      0.83        29
    법적 근거·조문       0.64      0.50      0.56        18
       절차·방법       0.75      0.60      0.67        15
     제재·구제수단       0.63      0.75      0.69        16
       증거·입증       0.59      0.68      0.63        19

    accuracy                           0.74       200
   macro avg       0.73      0.71      0.72       200
weighted avg       0.75      0.74      0.74       200

💾 /content/drive/MyDrive/intent_model_hybrid.joblib 저장 완료 ✅

🧠 [topic_name] 모델 학습 중...
✅ topic_name 정확도: 0.8050
                       precision    recall  f1-score   support

     부정행위 및 제3자 개입 책임       0.70      0.78   

In [31]:
examples = [
    "위자료 받을 수 있을까요?",
    "이혼 절차는 어떻게 되나요?",
    "배우자가 외도했어요.",
    "증거를 꼭 제출해야 하나요?",
    "위자료 금액은 어떻게 산정되나요?",
    "이혼 후에도 재산분할이 가능한가요?",
    "정서적 배신도 부정행위로 인정되나요?",
    "소송을 제기할 수 있는 기간이 있나요?",
     "남편이 외도했는데 위자료 받을 수 있나요?",
    "이혼 소송 절차는 어떻게 되나요?",
    "이혼 후 재산분할 청구 가능한가요?",
    "정서적 배신도 부정행위로 인정되나요?",
    "이혼 후에도 양육비를 받을 수 있나요?",
     "남편이 외도했는데 위자료 청구 가능한가요?",
    "이혼 소송 절차가 어떻게 진행되나요?",
    "위자료 금액은 어떤 기준으로 정하나요?",
    "이혼 후에도 재산분할 청구가 가능한가요?",
    "부정행위의 법적 근거는 무엇인가요?",
    "증거를 제출하지 못하면 어떻게 되나요?",
]

print("\n=== 🔍 통합 예측 결과 ===")
for q in examples:
    intent_pred = intent_pipe.predict([q])[0]
    topic_pred = topic_pipe.predict([q])[0]

    print(f"🗨️ 질문: {q}")
    print(f"➡️ 의도(law_intent): {intent_pred}")
    print(f"➡️ 주제(law_topic): {topic_pred}\n")



=== 🔍 통합 예측 결과 ===
🗨️ 질문: 위자료 받을 수 있을까요?
➡️ 의도(law_intent): 기타
➡️ 주제(law_topic): 위자료 및 손해배상 청구

🗨️ 질문: 이혼 절차는 어떻게 되나요?
➡️ 의도(law_intent): 절차·방법
➡️ 주제(law_topic): 부정행위의 법률적 근거

🗨️ 질문: 배우자가 외도했어요.
➡️ 의도(law_intent): 증거·입증
➡️ 주제(law_topic): 부정행위의 법률적 근거

🗨️ 질문: 증거를 꼭 제출해야 하나요?
➡️ 의도(law_intent): 증거·입증
➡️ 주제(law_topic): 부정행위의 법률적 근거

🗨️ 질문: 위자료 금액은 어떻게 산정되나요?
➡️ 의도(law_intent): 금액·산정
➡️ 주제(law_topic): 위자료 및 손해배상 청구

🗨️ 질문: 이혼 후에도 재산분할이 가능한가요?
➡️ 의도(law_intent): 기간·시효
➡️ 주제(law_topic): 이혼 시 금전 문제 (재산분할·양육비)

🗨️ 질문: 정서적 배신도 부정행위로 인정되나요?
➡️ 의도(law_intent): 개념·범위·기준
➡️ 주제(law_topic): 부정행위 및 제3자 개입 책임

🗨️ 질문: 소송을 제기할 수 있는 기간이 있나요?
➡️ 의도(law_intent): 기간·시효
➡️ 주제(law_topic): 이혼 시 금전 문제 (재산분할·양육비)

🗨️ 질문: 남편이 외도했는데 위자료 받을 수 있나요?
➡️ 의도(law_intent): 금액·산정
➡️ 주제(law_topic): 위자료 및 손해배상 청구

🗨️ 질문: 이혼 소송 절차는 어떻게 되나요?
➡️ 의도(law_intent): 절차·방법
➡️ 주제(law_topic): 부정행위의 법률적 근거

🗨️ 질문: 이혼 후 재산분할 청구 가능한가요?
➡️ 의도(law_intent): 금액·산정
➡️ 주제(law_topic): 이혼 시 금전 문제 (재산분할·양육비)

🗨️ 질문: 정서적 배신도 부정행위로 인정되나요?
➡️ 의도(law_